In [1]:
import open3d as o3d 
from visualization_utils import * 
import numpy as np 
from metrics.metrics_class import Metrics

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def intersect(pred_indices, gt_indices):
        intersection = np.intersect1d(pred_indices, gt_indices)
        return intersection.size / pred_indices.shape[0]

def iou(pred_indices, gt_indices):
        intersection = np.intersect1d(pred_indices, gt_indices)
        union = np.union1d(pred_indices, gt_indices)
        return intersection.size / union.size

def remove_semantics(labels,preds):
        gt_idcs = np.where(labels == 0)[0]
        new_ncuts_labels = preds.copy()
        for i in np.unique(preds):
                pred_idcs = np.where(preds == i)[0]
                cur_intersect = intersect(pred_idcs,gt_idcs)
                if cur_intersect > 0.8:
                        new_ncuts_labels[pred_idcs] = 0
        return new_ncuts_labels
        
def get_semantics(preds,gt_idcs,pcd):
        new_ncuts_labels = np.zeros_like(preds)
        new_pcd = o3d.geometry.PointCloud()
        new_pcd.points = pcd.points 
        num_clusters = 0 
        for i in np.unique(preds):
                #new_cols = np.zeros((np.asarray(pcd.points).shape[0],3))

                pred_idcs = np.where(preds == i)[0]
                cur_intersect = intersect(pred_idcs,gt_idcs)
                
                #new_cols[pred_idcs] = np.array([1,0,0])
                #new_pcd.colors = o3d.utility.Vector3dVector(new_cols)
                #o3d.visualization.draw_geometries([new_pcd])

                if cur_intersect > 0.1:
                        #print("intersect",cur_intersect)
                        new_ncuts_labels[pred_idcs] = 1
                        num_clusters += 1
        #o3d.visualization.draw_geometries([color_pcd_by_labels(pcd,new_ncuts_labels)])
        return np.where(new_ncuts_labels == 1)[0], num_clusters

def remove_semantics_pcd(pcd,labels,preds):
        cols = np.asarray(pcd.colors)
        gt_idcs = np.where(labels == 0)[0]
        new_ncuts_labels = preds.copy()
        for i in np.unique(preds):
                pred_idcs = np.where(preds == i)[0]
                cur_intersect = intersect(pred_idcs,gt_idcs)
                if cur_intersect > 0.8:
                        new_ncuts_labels[pred_idcs] = 0
                        cols[pred_idcs] = np.array([0,0,0])
                        
        pcd.colors = o3d.utility.Vector3dVector(cols)   
        return pcd

In [3]:
def remove_cols(pcd,cols,labels,unique_labels,idx=1): 

	new_pcd = o3d.geometry.PointCloud()
	new_pcd.points = pcd.points 
	new_cols = np.zeros((np.asarray(pcd.points).shape[0],3))
	pcd_cols = np.asarray(pcd.colors)
	
	idcs = np.where(labels == unique_labels[idx])[0]
	
	
	new_cols[idcs] = cols[idx]
	new_pcd.colors = o3d.utility.Vector3dVector(new_cols)
	#o3d.visualization.draw_geometries([new_pcd])
	return new_pcd,idcs 
	
	
def cluster_vis(pcd,cols,labels,unique_labels,idx=1): 

	new_pcd = o3d.geometry.PointCloud()
	new_pcd.points = pcd.points 
	new_cols = np.zeros((np.asarray(pcd.points).shape[0],3))
	pcd_cols = np.asarray(pcd.colors)
	
	idcs = np.where(labels == unique_labels[idx])[0]
	
	
	new_cols[idcs] = cols[idx]
	new_pcd.colors = o3d.utility.Vector3dVector(new_cols)
	#o3d.visualization.draw_geometries([new_pcd])
	return new_pcd,idcs 
	
	
	

In [4]:
out_folder = 'pcd_preprocessed/out_maps/'

fs = []
#fs.append('merge_a2_t3_0.05.pcd')
#fs.append('merge_a2_t3_0.025.pcd') 
#fs.append('merge_t3_0.05.pcd') 
#fs.append('ncuts_instances_tarl_spatial1.pcd') 
#fs.append('ncuts_sam1_instances.pcd')
#fs.append('ncuts_instances_tarl_spatial1.pcd')
#fs.append('hdbscan_instances.pcd') 
#fs.append('ncuts_a2_t3_0.025.pcd')
#fs.append('ncuts_tarl_sam_spatial.pcd')
#fs.append('ncuts_sam.pcd')
#fs.append('ncuts_tarl_norm.pcd')
fs.append(out_folder + 'ncuts_tarl_spatial1.pcd')
fs.append(out_folder + 'maskpls_5.pcd')

sem_gt = out_folder + "merge_part_kitti_semantic7.pcd"
inst_gt = out_folder + "merge_part_kitti_instance7.pcd"


sem_gt = 'out_kitti_semantic/000066.pcd'

pcd_gt = o3d.io.read_point_cloud(inst_gt)
pcd_sem = o3d.io.read_point_cloud(sem_gt)
unique_colors, labels_kitti = np.unique(np.asarray(pcd_gt.colors),axis=0, return_inverse=True)

o3d.visualization.draw_geometries([pcd_sem])

for f in fs : 
	print("file name",f)
	#pred_pcd = o3d.io.read_point_cloud(f)
	pred_pcd = o3d.io.read_point_cloud('test_data/000066.pcd')
	#o3d.visualization.draw_geometries([pred_pcd])
	unique_colors, labels_ncuts = np.unique(np.asarray(pred_pcd.colors), axis=0, return_inverse=True)
	metrics_pred = Metrics(name=f)
	#new_labels = remove_semantics(labels_kitti,labels_ncuts)	
	#metrics_pred.update_stats(new_labels,labels_kitti,calc_all=False,calc_lstq=False)
	metrics_pred.semantic_eval(pred_pcd,pcd_sem)

[Open3D WARNING] Read PCD failed: unable to open file: out_kitti_semantic/000066.pcd
file name pcd_preprocessed/out_maps/ncuts_tarl_spatial1.pcd
[Open3D WARNING] Read PCD failed: unable to open file: test_data/000066.pcd
Metrics for file pcd_preprocessed/out_maps/ncuts_tarl_spatial1.pcd
file idx 1
Semantics for class car


IndexError: list index out of range